In [1]:
import json

In [63]:
def clean(value, ftype):
    if ftype == 'phone':
        clean_value = ''
        for s in value:
            if s.isdigit():
                clean_value += s
    else:
        clean_value = value
        
    return clean_value

In [75]:
translate_feature = {
    'phone': 'Phone Number',
    'email': 'Email',
    'price': 'Price',
    'height': 'Height',
    'weight': 'Weight',
    'age': 'Age',
    'name': 'Name',
    'location': 'Location',
    'hair_color': 'Hair Color',
    'services': 'Services',
    'ethnicity': 'Nationality / Ethnicity'
}

In [3]:
data_file = '../cluster_annotations/cluster_annotations.json'
f = open(data_file, 'r')
data = eval(f.read())
f.close()

questions_file = '../../questions/pure_agg_gt_questions_V3.json'
questions = []
with open(questions_file, 'r') as f:
    for line in f:
        temp = json.loads(line)
        questions.append(temp)

### Create huge list of ads

In [4]:
ads = []
for clus in data:
    for ad in clus:
        ads.append(ad)

### Determine Filter and Query Features and Values

In [79]:
filter_criteria = {}
for question in questions:
    filter_criteria[question['id']] = {}
    qlines = question['SPARQL'][1].split('\n')
    filter_criteria[question['id']]['query'] = qlines[1].split('?')[1].strip()
    filter_criteria[question['id']]['filters'] = {}
    for line in qlines:
        if 'qpr:' in line and '?' not in line and 'PREFIX' not in line:
            filter_type = line.split(' ')[0].replace('\tqpr:', '')
            filter_value = line.split("'")[1].replace("'", '')
            filter_criteria[question['id']]['filters'][filter_type] = filter_value

In [84]:
pure_agg_gt_answers = {}
for quest in filter_criteria.keys():
    pure_agg_gt_answers[quest] = {}
    matches = []
    query = filter_criteria[quest]['query']
    for ad in ads:
        match = 0
        if 'features' in ad.keys():
            for feature in filter_criteria[quest]['filters'].keys():
                if translate_feature[feature] in ad['features'].keys():
                    for entry in ad['features'][translate_feature[feature]]:
                        if 'text' in entry.keys():
                            ad_value = clean(entry['text'], feature)
                            if filter_criteria[quest]['filters'][feature].lower() in ad_value.lower():
                                match += 1
        # If you found as many matches as there are filters
        if match == len(filter_criteria[quest]['filters'].keys()):
            matches.append(ad)
    
    # Now take query value from each matching ad
    for ad in matches:
        query_values = []
        if translate_feature[query] in ad['features'].keys():
            for entry in ad['features'][translate_feature[query]]:
                if 'text' in entry.keys():
                    query_values.append(entry['text'])
        pure_agg_gt_answers[quest][ad['obj_parent']] = query_values

## Save Ad IDs to be used for Average Precision Calculation

This will be manually renamed to avoid overwriting

In [96]:
pure_agg_gt_quest2ads = {}
for quest in pure_agg_gt_answers.keys():
    pure_agg_gt_quest2ads[int(quest)] = pure_agg_gt_answers[quest].keys()

filename = '../posted_gt/gt_answer_key/pure_agg_gt_quest2ads_rewrite.json'
with open(filename, 'w') as outfile:
    json.dump(pure_agg_gt_quest2ads, outfile)

## Save Extracted Query Values to be Manually Normalized

This will be manually renamed to avoid overwriting

In [100]:
# Make list of all types to be normalized
qtypes = []
for quest in filter_criteria.keys():
    qtypes.append(filter_criteria[quest]['query'])
    qtypes = list(set(qtypes))

In [104]:
norm_values = {}
for query in qtypes:
    norm_values[query] = {}
    qvalues = []
    for quest in filter_criteria.keys():
        if filter_criteria[quest]['query'] == query:
            for ad in pure_agg_gt_answers[quest].keys():
                qvalues.extend(pure_agg_gt_answers[quest][ad])
    qvalues = list(set(qvalues))
    # Make empty dictionary entry to be manually (normalized) populated later
    for value in qvalues:
        norm_values[query][value] = ' '

In [107]:
filename = 'norm_pure_agg_gt_values_rewrite.json'
with open(filename, 'w') as outfile:
    json.dump(norm_values, outfile, indent=2)

## Save Incomplete Answer Key to be Populated by Next Script

This will be manually renamed to avoid overwriting

Trying to make the format match agg_answer_key.json

In [139]:
answer_key = {}
for quest in questions:
    answer_key[str(quest['id'])] = {}
    answer_key[str(quest['id'])]['question'] = quest['question']
    answer_key[str(quest['id'])]['SPARQL'] = quest['SPARQL']
    answer_key[str(quest['id'])]['type'] = quest['type']
    answer_key[str(quest['id'])]['id'] = str(quest['id'])
    answer_key[str(quest['id'])]['agg_answer'] = ' '
    extractions = []
    for ad in pure_agg_gt_answers[quest['id']]:
        extractions.extend(pure_agg_gt_answers[quest['id']][ad])
    answer_key[str(quest['id'])]['extractions'] = extractions
    answer_key[str(quest['id'])]['feature'] = filter_criteria[quest['id']]['query']

In [140]:
filename = 'pure_agg_gt_answer_key_blank.json'
with open(filename, 'w') as outfile:
    json.dump(answer_key, outfile, indent=2)

In [138]:
filter_criteria[1]

{'filters': {u'phone': u'4388042437'}, 'query': u'name'}